In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler

In [6]:
dtr = pd.read_csv('diabetes.csv')

In [7]:
dtr.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
dtr.shape

(768, 9)

In [9]:
x=dtr.iloc[:,:-1].values
y=dtr.iloc[:,-1].values


In [10]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
print(x)
print(y)

[[0.35294118 0.74371859 0.59016393 ... 0.50074516 0.23441503 0.48333333]
 [0.05882353 0.42713568 0.54098361 ... 0.39642325 0.11656704 0.16666667]
 [0.47058824 0.91959799 0.52459016 ... 0.34724292 0.25362938 0.18333333]
 ...
 [0.29411765 0.6080402  0.59016393 ... 0.390462   0.07130658 0.15      ]
 [0.05882353 0.63316583 0.49180328 ... 0.4485842  0.11571307 0.43333333]
 [0.05882353 0.46733668 0.57377049 ... 0.45305514 0.10119556 0.03333333]]
[1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1
 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0
 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1
 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state = 120)


In [12]:
x_train = X_train.T
y_train = y_train.reshape(1, y_train.shape[0])
x_test = X_test.T
y_test = y_test.reshape(1, y_test.shape[0])

In [13]:
input_layer=x_train.shape[0]
hidden_layer=12
output_layer=1


In [14]:
np.random.seed(1) 
W1 = np.random.uniform(-1,1,size=(hidden_layer, input_layer))
b1 = np.zeros((hidden_layer, 1))
W2 = np.random.uniform(-1,1,size=(output_layer, hidden_layer))
b2 = np.zeros((output_layer, 1))

In [15]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [16]:
def forward_propagation(W1,x_train,b1,W2,b2):
    
    Z1 = np.dot(W1, x_train) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    cache = {"Z1": Z1,"A1": A1,"Z2": Z2,"A2": A2}
    return A2, cache

In [31]:
def cost_main(A2,y_train):
    
    m = y_train.shape[1] 
    cost= np.multiply(np.log(A2), y_train) + np.multiply((1-y_train), np.log(1 - A2))
    cost = - np.sum(cost) / m
    cost = float(np.squeeze(cost))
    return cost
                   

In [36]:
def backward_propagation(W1,W2, cache, x_train, y_train):
    m = x_train.shape[1]
    A1 = cache['A1']
    A2 = cache['A2']
   
    dZ2 = A2-y_train
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, x_train.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2,"db2": db2}
    
    return grads


In [37]:
vdw1= np.zeros((W1.shape[0], W1.shape[1]))
vdw2= np.zeros((W2.shape[0], W2.shape[1]))
vdb1= np.zeros((b1.shape[0], b1.shape[1]))
vdb2= np.zeros((b2.shape[0], b2.shape[1]))

In [38]:
def gradient_descent(W1,b1,W2,b2, grads, learning_rate ,vdw1,vdw2,vdb1,vdb2,beta):
   
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    vdw1=beta*vdw1+(1-beta)*dW1
    vdw2=beta*vdw2+(1-beta)*dW2
    vdb1=beta*vdb1+(1-beta)*db1
    vdb2=beta*vdb2+(1-beta)*db2
    
    W1 = W1 - learning_rate * vdw1
    b1 = b1 - learning_rate * vdb1
    W2 = W2 - learning_rate * vdw2
    b2 = b2 - learning_rate * vdb2
    
    
    return W1,W2,b1,b2,vdw1,vdw2,vdb1,vdb2

In [39]:
def neural_network_model(x_train, y_train,num_iterations,W1,W2,b1,b2,vdw1,vdw2,vdb1,vdb2,learning_rate,beta):
    
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(W1,x_train,b1,W2,b2)
        cost =cost_main(A2,y_train)
        grads = backward_propagation(W1,W2, cache, x_train, y_train)
        W1,W2,b1,b2,vdw1,vdw2,vdb1,vdb2=gradient_descent(W1,b1,W2,b2, grads, learning_rate ,vdw1,vdw2,vdb1,vdb2,beta)
        if i % 5 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    return W1,W2,b1,b2

In [48]:
W1,W2,b1,b2=neural_network_model(x_train, y_train,3000,W1,W2,b1,b2,vdw1,vdw2,vdb1,vdb2,0.01,0.98)

Cost after iteration 0: 0.518281
Cost after iteration 5: 0.518274
Cost after iteration 10: 0.518256
Cost after iteration 15: 0.518228
Cost after iteration 20: 0.518192
Cost after iteration 25: 0.518148
Cost after iteration 30: 0.518097
Cost after iteration 35: 0.518039
Cost after iteration 40: 0.517976
Cost after iteration 45: 0.517907
Cost after iteration 50: 0.517834
Cost after iteration 55: 0.517756
Cost after iteration 60: 0.517675
Cost after iteration 65: 0.517590
Cost after iteration 70: 0.517502
Cost after iteration 75: 0.517411
Cost after iteration 80: 0.517318
Cost after iteration 85: 0.517223
Cost after iteration 90: 0.517126
Cost after iteration 95: 0.517027
Cost after iteration 100: 0.516927
Cost after iteration 105: 0.516825
Cost after iteration 110: 0.516722
Cost after iteration 115: 0.516618
Cost after iteration 120: 0.516513
Cost after iteration 125: 0.516407
Cost after iteration 130: 0.516300
Cost after iteration 135: 0.516193
Cost after iteration 140: 0.516086
Cost af

Cost after iteration 1390: 0.494596
Cost after iteration 1395: 0.494533
Cost after iteration 1400: 0.494470
Cost after iteration 1405: 0.494407
Cost after iteration 1410: 0.494345
Cost after iteration 1415: 0.494282
Cost after iteration 1420: 0.494220
Cost after iteration 1425: 0.494158
Cost after iteration 1430: 0.494096
Cost after iteration 1435: 0.494034
Cost after iteration 1440: 0.493973
Cost after iteration 1445: 0.493911
Cost after iteration 1450: 0.493850
Cost after iteration 1455: 0.493788
Cost after iteration 1460: 0.493727
Cost after iteration 1465: 0.493666
Cost after iteration 1470: 0.493605
Cost after iteration 1475: 0.493545
Cost after iteration 1480: 0.493484
Cost after iteration 1485: 0.493424
Cost after iteration 1490: 0.493363
Cost after iteration 1495: 0.493303
Cost after iteration 1500: 0.493243
Cost after iteration 1505: 0.493183
Cost after iteration 1510: 0.493124
Cost after iteration 1515: 0.493064
Cost after iteration 1520: 0.493005
Cost after iteration 1525: 0

Cost after iteration 2750: 0.481711
Cost after iteration 2755: 0.481676
Cost after iteration 2760: 0.481641
Cost after iteration 2765: 0.481606
Cost after iteration 2770: 0.481571
Cost after iteration 2775: 0.481536
Cost after iteration 2780: 0.481501
Cost after iteration 2785: 0.481467
Cost after iteration 2790: 0.481432
Cost after iteration 2795: 0.481397
Cost after iteration 2800: 0.481363
Cost after iteration 2805: 0.481329
Cost after iteration 2810: 0.481294
Cost after iteration 2815: 0.481260
Cost after iteration 2820: 0.481226
Cost after iteration 2825: 0.481192
Cost after iteration 2830: 0.481158
Cost after iteration 2835: 0.481124
Cost after iteration 2840: 0.481090
Cost after iteration 2845: 0.481056
Cost after iteration 2850: 0.481022
Cost after iteration 2855: 0.480988
Cost after iteration 2860: 0.480955
Cost after iteration 2865: 0.480921
Cost after iteration 2870: 0.480888
Cost after iteration 2875: 0.480854
Cost after iteration 2880: 0.480821
Cost after iteration 2885: 0

In [49]:
def prediction(W1,W2,b1,b2,x_train):
    A2, cache = forward_propagation(W1,x_train,b1,W2,b2)
    predictions = np.round(A2)
    
    return predictions

In [50]:
predictions =prediction(W1,W2,b1,b2,x_train)
print ('Accuracy Train: %d' % float((np.dot(y_train, predictions.T) + np.dot(1 - y_train, 1 - predictions.T))/float(y_train.size)*100) + '%')
predictions = prediction(W1,W2,b1,b2,x_test)
print ('Accuracy Test: %d' % float((np.dot(y_test, predictions.T) + np.dot(1 - y_test, 1 - predictions.T))/float(y_test.size)*100) + '%')

Accuracy Train: 77%
Accuracy Test: 74%
